In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
protoFile = "models/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "models/pose_iter_160000.caffemodel"

In [3]:
nPoints = 15
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
pose_net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

MPII Output Format Head – 0, Neck – 1, Right Shoulder – 2, Right Elbow – 3, Right Wrist – 4, Left Shoulder – 5, Left Elbow – 6, Left Wrist – 7, Right Hip – 8, Right Knee – 9, Right Ankle – 10, Left Hip – 11, Left Knee – 12, Left Ankle – 13, Chest – 14, Background – 15

In [4]:
def pose_estimation(frame):
    blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (368, 368), (0, 0, 0), swapRB=False, crop=False)
    pose_net.setInput(blob)
    output = pose_net.forward()

    height, width, _ = frame.shape
    scaleX = width / output.shape[3]
    scaleY = height / output.shape[2]

    # Stroe keypoints
    points = []

    threshold = 0.1

    for i in range(nPoints):
        
        probMap = output[0, i, :, :]
        _, prob, _, point = cv2.minMaxLoc(probMap)

        x = scaleX * point[0]
        y = scaleY * point[1]

        if prob > threshold : 
            points.append((int(x), int(y)))
        else :
            points.append(None)

    imSkeleton = frame.copy()
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(imSkeleton, points[partA], points[partB], (0, 255,0), 2)
            cv2.circle(imSkeleton, points[partA], 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

    return points, imSkeleton

In [9]:
import math

def calculate_angle(point1, point2, point3):
    vector1 = [point1[0] - point2[0], point1[1] - point2[1]]
    vector2 = [point3[0] - point2[0], point3[1] - point2[1]]

    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
    magnitude1 = math.sqrt(vector1[0] ** 2 + vector1[1] ** 2)
    magnitude2 = math.sqrt(vector2[0] ** 2 + vector2[1] ** 2)
    
    try:
        cosine_angle = dot_product / (magnitude1 * magnitude2)
    except Exception as e : 
        return False , 0
    
    angle_radians = math.acos(cosine_angle)
    angle_degrees = math.degrees(angle_radians)

    return True, angle_degrees

def recognize_posture(points):
    right_shoulder = points[2]
    right_elbow = points[3]
    right_wrist = points[4]
    left_shoulder = points[5]
    left_elbow = points[6]
    left_wrist = points[7]
    right_hip = points[8]
    left_hip = points[11]
    right_knee = points[9]
    left_knee = points[12]
    right_ankle = points[10]
    left_ankle = points[13]

    ok, right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    ok, left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    
    if not ok:
        return "unknown _ Error"
    elif right_knee_angle < 155 and left_knee_angle < 155:
        return "sitting"
    elif 155 <= right_knee_angle and 155 <= left_knee_angle:
        return "standing"
    else:
        return "unknown"

In [10]:
input_ = "media/videos/_sitting-down-on.mp4"
video_cap = cv2.VideoCapture(input_)
ok, frame = video_cap.read()

pvs_pts, img = pose_estimation(frame)

if not video_cap.isOpened():
    print("Could not open video")
else : 
    width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
name = 'Pose_recognition.avi'
video_out = cv2.VideoWriter(name, cv2.VideoWriter_fourcc(*'XVID'), 20, (width, height))

while True:
    ret, frame = video_cap.read()
    if not ret:
        break 
    pts, imframe = pose_estimation(frame)
    
    for i in range(len(pts)):
        if pts[i] == None : 
            pts[i] = pvs_pts[i]
            
    pos = recognize_posture(pts)
    
    cv2.putText(imframe, "Posture : " + pos, (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 220, 220), 2)
    video_out.write(imframe)
    
video_cap.release()
video_out.release()